In [1]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import pandas as pd
import librosa as lb
from torch import nn
from argparse import Namespace
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from kymatio.numpy import Scattering1D
from torch import from_numpy
import torch
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassF1Score
from torchmetrics.classification import MulticlassAccuracy
from torchmetrics.classification import MulticlassPrecision
from torchmetrics.classification import MulticlassConfusionMatrix
import matplotlib.pyplot as plt
import seaborn as sns
import io
from PIL import Image
from torchvision.transforms import ToTensor
from lightning.pytorch import seed_everything
seed_everything(42)
import time

Global seed set to 42


## Data segment

In [2]:
def segment(data_path, audfile_dir, dest_fs, test_files=[], seg_time=0.09, hog_time=0.05):
    data = pd.read_csv(data_path)
    files = data['New_Filename'].values
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    for name in files:
        aud, fs = lb.load(os.path.join(audfile_dir, name), sr=dest_fs)
        lbl = np.int16(data[data['New_Filename'] == name]['label'].values[0])
        seg_size = int(seg_time * fs);
        hog_size = int(hog_time * fs);
        if name in test_files:
            for i in range(0, len(aud)-hog_size, hog_size):
                if len(aud) >= i+seg_size:
                    X_test.append(aud[i:i+seg_size])
                    Y_test.append(lbl)
        else:
            for i in range(0, len(aud)-hog_size, hog_size):
                if len(aud) >= i+seg_size:
                    X_train.append(aud[i:i+seg_size])
                    Y_train.append(lbl)
    return np.array(X_train), np.array(Y_train), np.array(X_test), np.array(Y_test)

## Dataset

In [3]:
class shipseardataset(Dataset):
    def __init__(self, data_path, test_files=[], mode ='train', val_tr_pat=0.15, 
                 test_part=0.9, sample_rate=51200, win_time=0.1, step_time=0.05, 
                 feature_param={'J':6, 'Q':8}):
        audfile_dir = os.path.dirname(data_path)        
        X_train, Y_train, X_test, Y_test = segment(data_path, audfile_dir, sample_rate, test_files, win_time, step_time)
        if len(test_files)==0:
            X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=test_part,random_state=6)
            
        X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=val_tr_pat,random_state=6)
        N = X_train.shape[1]
        self.scattering = Scattering1D(feature_param['J'], N, feature_param['Q'])
        
        if mode == 'train':
            self.x = X_train
            self.y = Y_train
        elif mode == 'val':
            self.x = X_val
            self.y = Y_val
        else:
            self.x = X_test
            self.y = Y_test
    def __getitem__(self, i):
        x = self.x[i] # BN X N
        x = self.scattering(x)[np.newaxis]
        #x = lb.feature.melspectrogram(y=x,hop_length=501)
        #x = lb.power_to_db(x,ref=np.max)[np.newaxis]  # 1 X P X N/(2**J), P=1+JQ+J(J-1)Q/2.
        y = self.y[i] # BN
        return x, y
    
    def __len__(self):
        return len(self.y)

## Network

In [4]:

##---------- Spatial Attention ----------
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0,dilation=1, groups=1, relu=True, bn=True, bias=True):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class spatial_attn_layer(nn.Module):
    def __init__(self, kernel_size=5):
        super(spatial_attn_layer, self).__init__()
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1)//2, relu=False)
    def forward(self, x):
        # import pdb;pdb.set_trace()
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = torch.sigmoid(x_out) # broadcasting
        return x * scale


##########################################################################
## ------ Channel Attention --------------
class ca_layer(nn.Module):
    def __init__(self, channel, reduction=8, bias=True):
        super(ca_layer, self).__init__()
        # global average pooling: feature --> point
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # feature channel downscale and upscale --> channel weight
        self.conv_du = nn.Sequential(
                nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=bias),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=bias),
                nn.Sigmoid()
        )

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv_du(y)
        return x * y


class ResBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResBlock, self).__init__()
        #这里定义了残差块内连续的2个卷积层
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            #shortcut，这里为了跟2个卷积层的结果结构一致，要做处理
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
        self.ca = ca_layer(outchannel)
        self.sa = spatial_attn_layer()
        
    def forward(self, x):
        out = self.left(x)
        out = self.ca(out)
        out = self.sa(out)
        #将2个卷积层的输出跟处理过的x相加，实现ResNet的基本结构
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out

class network(nn.Module):
    def __init__(self, ResBlock, in_channel=1, out_class=5):
        super(network, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channel, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResBlock, 256, 2, stride=2)        
        self.layer4 = self.make_layer(ResBlock, 512, 2, stride=2)   
        self.drop = nn.Dropout(p=0.2)
        self.avg = torch.nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, out_class)
    #这个函数主要是用来，重复同一个残差块    
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        #在这里，整个ResNet18的结构就很清晰了
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.drop(out)
        out = self.layer3(out)
        out = self.drop(out)
        out = self.layer4(out)
        out = self.drop(out)
        out = self.avg(out)
        out = torch.squeeze(out)
        out = self.fc(out)
        return out




## PLmodel

In [5]:

class Model(pl.LightningModule):
    def __init__(self, hparams):
        super(Model, self).__init__()
        
        self.save_hyperparameters(hparams)
        
        self.net = network(ResBlock,in_channel=1,out_class=5)
        print(self.net)        
        self.softmax = torch.nn.Softmax(dim=1)
        self.loss = torch.nn.CrossEntropyLoss()  # DiceLoss()
        self.accuracy = MulticlassAccuracy(num_classes=5)
        self.f1_score = MulticlassF1Score(num_classes=5)
        self.precious = MulticlassPrecision(num_classes=5)
        self.accuracy1 = MulticlassAccuracy(num_classes=5,average='none')
        self.f1_score1 = MulticlassF1Score(num_classes=5,average='none')
        self.precious1 = MulticlassPrecision(num_classes=5,average='none')
        self.confuse = MulticlassConfusionMatrix(num_classes=5)
        self.val_acc = 0
        self.global_confuse = torch.zeros(5,5,device='cuda:0')

    def forward(self, x):
        # called with self(x)
        return self.net(x)

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x = batch[0].float() # NB X N
        y = batch[1] # NB
        y_hat = self(x)
        loss = self.loss(y_hat, y.long())
        self.accuracy.update(torch.argmax(y_hat,dim=1), y)
        metric = self.accuracy.compute()        
        # loss_all = loss_Dice
        self.log("train/acc", metric, on_step=False, on_epoch=True)
        self.log("train/total_loss", loss)
        self.accuracy.reset()
        
        return {'loss': loss}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x = batch[0].float() # NB X N
        y = batch[1] # NB
        y_hat = self(x)
        self.accuracy.update(torch.argmax(y_hat,dim=1), y)
        metric = self.accuracy.compute()
        self.log("val_acc", metric, on_step=False, on_epoch=True)
        self.accuracy.reset()
        
    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x = batch[0].float() # NB X N
        y = batch[1] # NB
        y_hat = self(x)

        self.accuracy.update(torch.argmax(y_hat,dim=1), y)
        metric = self.accuracy.compute()
        self.log("test_acc", metric, on_step=False, on_epoch=True)
        self.accuracy.reset()

        self.f1_score.update(torch.argmax(y_hat,dim=1), y)
        f1= self.f1_score.compute()
        self.log("f1_score", f1, on_step=False, on_epoch=True)
        self.f1_score.reset()

        self.precious.update(torch.argmax(y_hat,dim=1), y)
        precious= self.precious.compute()
        self.log("precious", precious, on_step=False, on_epoch=True)
        self.precious.reset()

        self.accuracy1.update(torch.argmax(y_hat,dim=1), y)
        metric = self.accuracy1.compute()  
        for i,acc3 in enumerate(metric):      
            self.log(f"test_acc-{i}", acc3, on_step=False, on_epoch=True)
        self.accuracy1.reset()

        self.f1_score1.update(torch.argmax(y_hat,dim=1), y)
        f12= self.f1_score1.compute() 
        for i,acc4 in enumerate(f12):       
            self.log(f"f1_score-{i}", acc4, on_step=False, on_epoch=True)
        self.f1_score1.reset()

        self.precious1.update(torch.argmax(y_hat,dim=1), y)
        precious2= self.precious1.compute() 
        for i,acc5 in enumerate(precious2):       
            self.log(f"precious-{i}", acc5, on_step=False, on_epoch=True)
        self.precious1.reset()

        self.confuse.update(torch.argmax(y_hat,dim=1), y)
        confuse = self.confuse.compute()
        self.global_confuse += confuse
        print(self.global_confuse)
        self.confuse.reset()

        

        
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x = batch[0].float() # NB X N
        y_hat = self(x)
        res = torch.argmax(y_hat, dim=1)
        
        return res


    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        self.logger.log_hyperparams(self.hparams)

#         optimizer = torch.optim.SGD(self.net.parameters(), lr=self.hparams.lr, momentum=0.9)

        optimizer = torch.optim.Adam(self.net.parameters(), lr=self.hparams.lr,
                                     betas=(self.hparams.b1, self.hparams.b2))
        lr_scheduler = {'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.2, threshold=1e-4, patience=3, eps=0.001),
            'interval': 'epoch',
                        'monitor': 'val_acc'}
#         lr_scheduler = {'scheduler': torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=0.01),
#                         'interval': 'batch'}
#         lr_scheduler = {'scheduler': torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5),
#                         'interval': 'epoch'}
        return [optimizer], [lr_scheduler]

    def train_dataloader(self):
        # REQUIRED
        train_dataset = shipseardataset(self.hparams.data_path, self.hparams.test_files, 'train', 
                                        val_tr_pat=self.hparams.var_par, test_part=self.hparams.test_par, 
                                        win_time=self.hparams.win_time, sample_rate=self.hparams.sample_rate, 
                                        step_time=self.hparams.step_time, 
                                        feature_param=self.hparams.feature_param)
        return DataLoader(train_dataset, shuffle=True, pin_memory=True,
                          batch_size=self.hparams.batch_size, num_workers=int(self.hparams.num_threads))

    def val_dataloader(self):
        # OPTIONAL
        val_dataset = shipseardataset(self.hparams.data_path, self.hparams.test_files, 'val', 
                                        val_tr_pat=self.hparams.var_par, test_part=self.hparams.test_par, 
                                        win_time=self.hparams.win_time, sample_rate=self.hparams.sample_rate, 
                                        step_time=self.hparams.step_time, 
                                        feature_param=self.hparams.feature_param)
        return DataLoader(val_dataset, shuffle=False, pin_memory=True,
                          batch_size=self.hparams.batch_size, num_workers=int(self.hparams.num_threads))

    def test_dataloader(self):
        # OPTIONAL
        test_data = shipseardataset(self.hparams.data_path, self.hparams.test_files, 'test', 
                                        val_tr_pat=self.hparams.var_par, test_part=self.hparams.test_par, 
                                        win_time=self.hparams.win_time, sample_rate=self.hparams.sample_rate, 
                                        step_time=self.hparams.step_time, 
                                        feature_param=self.hparams.feature_param)
        return DataLoader(test_data, shuffle=False, pin_memory=True,
                          batch_size=64, num_workers=int(self.hparams.num_threads))

## Super-Paramter

In [6]:
parameters= {
    'feature_param':{'J':6, 'Q':8},
    'data_path': '/home/ubuntu/KFT/Data/underwater/shipsEar_AUDIOS/shipsEar.csv',
    'var_par': 0.15,
    'test_par': 0.9,
    'sample_rate': 51200,
    'win_time': 0.5,
    'step_time': 0.3,
    'in_channel':1,
    'out_class':5,
    'results_dir':'/home/ubuntu/KFT/Data/underwater/shipsEar_AUDIOS/result',
    'results_name': 'model-0.9-scat',
    'version':2,
    'port':8099,
    'batch_size': 32,
    'num_threads': 4,
    'lr': 0.001,
    'b1': 0.9,
    'b2': 0.999,
    'max_epochs': 200,
    'test_files': [],
}

## PLTraining

In [7]:
hyparams = Namespace(**parameters)

plmodel = Model(hyparams)

earlystop = pl.callbacks.EarlyStopping('val_acc', mode='max', min_delta=1e-4, patience=8)
tb_logger = pl.loggers.TensorBoardLogger(hyparams.results_dir, hyparams.results_name, version=hyparams.version)
checkpoint_callback = pl.callbacks.ModelCheckpoint(tb_logger.log_dir, monitor='val_acc', mode='max',
                                                  verbose=True)
# lr_logger = pl.callbacks.lr_logger.LearningRateLogger()
cmd = 'tensorboard --logdir %s --port %s' % (tb_logger.log_dir, hyparams.port)
print(cmd)

gpus = [0]
tb_logger.log_hyperparams(hyparams)
trainer = pl.Trainer( accelerator ='gpu', devices = 1,
                     logger=tb_logger, max_epochs=hyparams.max_epochs, min_epochs=10,
                     callbacks=[checkpoint_callback, earlystop])

# checkpoint_dir = '/nas/home/fkong/data/farimage/simulation/fracture_new/preds/frac_multi/UNET/version_3/'
# ckpt_name = [x for x in os.listdir(checkpoint_dir) if '.ckpt' in x][0]

# segmodel = UnetModel.load_from_checkpoint(
#     os.path.join(checkpoint_dir, ckpt_name),
#     map_location=None)
# segmodel.current_epoch = 0
now = time.time()
trainer.fit(plmodel)
finish = time.time()
print(finish-now)
# trainer.test(segmodel)

# checkpoint_dir = os.path.join(tb_logger.log_dir)
# ckpt_name = [x for x in os.listdir(checkpoint_dir) if '.ckpt' in x][0]

# segmodel = UnetModel.load_from_checkpoint(
#     os.path.join(checkpoint_dir, ckpt_name),
#     map_location=None, hparams=hyparams)

trainer.test(plmodel)

network(
  (conv1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
      (ca): ca_layer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(64, 8, kernel_size=(1, 1), stride=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(8, 64, kernel_size=(1, 1), stride=(1, 1

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/ubuntu/miniconda3/envs/liu/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:634: Checkpoint directory /home/ubuntu/KFT/Data/underwater/shipsEar_AUDIOS/result/model-0.9-scat/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | net       | network                   | 11.3 M
1 | softmax   | Softmax                

Epoch 0: 100%|██████████| 101/101 [01:38<00:00,  1.02it/s, v_num=1]        

Epoch 0, global step 101: 'val_acc' reached 0.32206 (best 0.32206), saving model to '/home/ubuntu/KFT/Data/underwater/shipsEar_AUDIOS/result/model-0.9-scat/version_1/epoch=0-step=101.ckpt' as top 1


Epoch 1: 100%|██████████| 101/101 [01:40<00:00,  1.01it/s, v_num=1]

...

Epoch 36: 100%|██████████| 101/101 [01:37<00:00,  1.04it/s, v_num=1]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


4244.04504776001
Testing DataLoader 0:   0%|          | 0/530 [00:00<?, ?it/s]tensor([[11.,  0.,  0.,  0.,  0.],
        [ 1.,  5.,  0.,  0.,  0.],
        [ 1.,  0., 22.,  0.,  0.],
        [ 0.,  0.,  0., 22.,  0.],
        [ 0.,  0.,  0.,  0.,  2.]], device='cuda:0')
Testing DataLoader 0:   0%|          | 1/530 [00:00<03:06,  2.83it/s]tensor([[23.,  0.,  1.,  0.,  0.],
        [ 1., 10.,  0.,  0.,  0.],
        [ 1.,  2., 43.,  1.,  0.],
        [ 0.,  0.,  0., 37.,  0.],
        [ 0.,  0.,  0.,  0.,  9.]], device='cuda:0')
Testing DataLoader 0:   0%|          | 2/530 [00:00<02:12,  3.98it/s]tensor([[32.,  2.,  1.,  0.,  0.],
        [ 1., 18.,  1.,  0.,  0.],
        [ 1.,  2., 67.,  2.,  0.],
        [ 0.,  0.,  0., 53.,  0.],
        [ 0.,  0.,  0.,  0., 12.]], device='cuda:0')
Testing DataLoader 0:   1%|          | 3/530 [00:00<01:53,  4.65it/s]tensor([[44.,  2.,  1.,  0.,  0.],
        [ 1., 28.,  3.,  0.,  0.],
        [ 2.,  2., 92.,  2.,  0.],
        [ 0.,  0.,  0., 60.,  0

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         f1_score          │     0.925947368144989     │
│        f1_score-0         │    0.9012397527694702     │
│        f1_score-1         │    0.8665128946304321     │
│        f1_score-2         │     0.94816654920578      │
│        f1_score-3         │    0.9450876712799072     │
│        f1_score-4         │    0.9687251448631287     │
│         precious          │    0.9293356537818909     │
│        precious-0         │    0.9425467848777771     │
│        precious-1         │    0.8635663986206055     │
│        precious-2         │    0.9411125183105469     │
│        precious-3         │    0.9531462788581848     │
│        precious-4         │    0.9463019371032715     │
│         test_acc          │    0.9305164217948914     │
│        test_acc-0         │    0.8729932904243469     │
│        test_acc-1         │    0.8834609389305115     │
│        test_acc-2         │    0.9572166204452515     │
│        test_acc-3         │     0.940555989742279     │
│        test_acc-4         │    0.9983493685722351     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9305164217948914,
  'f1_score': 0.925947368144989,
  'precious': 0.9293356537818909,
  'test_acc-0': 0.8729932904243469,
  'test_acc-1': 0.8834609389305115,
  'test_acc-2': 0.9572166204452515,
  'test_acc-3': 0.940555989742279,
  'test_acc-4': 0.9983493685722351,
  'f1_score-0': 0.9012397527694702,
  'f1_score-1': 0.8665128946304321,
  'f1_score-2': 0.94816654920578,
  'f1_score-3': 0.9450876712799072,
  'f1_score-4': 0.9687251448631287,
  'precious-0': 0.9425467848777771,
  'precious-1': 0.8635663986206055,
  'precious-2': 0.9411125183105469,
  'precious-3': 0.9531462788581848,
  'precious-4': 0.9463019371032715}]

In [8]:
# test_data = shipseardataset(hyparams.data_path, hyparams.test_files, 'test')
# test_loader = DataLoader(test_data)

# trainer.test(plmodel,dataloaders=test_loader)

In [9]:
# !rm -rf /home/ubuntu/KFT/Data/underwater/shipsEar_AUDIOS/result/